In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
print(torch.__version__)

1.8.2


In [2]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

batch_size = 32
dataloader = DataLoader(training_data, batch_size=batch_size, num_workers=4)

Files already downloaded and verified


In [3]:
for X, y in dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 3, 32, 32])
Shape of y: torch.Size([32]) torch.int64


In [4]:
import torchvision.models as models
num_classes=10
model = models.resnet50(num_classes=num_classes).to('cuda:0')

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [5]:
def train(model):
    model.train(True)

    for i, (inputs, labels) in enumerate(dataloader):
        inputs,labels=inputs.to('cuda'),labels.to('cuda')

        # run forward pass
        optimizer.zero_grad()
        outputs = model(inputs)

        # run backward pass
        loss_fn(outputs, labels).backward()
        optimizer.step()
        
        print('\r',i,end='')

In [6]:
for i, (inputs, labels) in enumerate(dataloader):
    if i > 500: break
    # run forward pass
    optimizer.zero_grad()
    outputs = model(inputs.to('cuda:0'))

    # run backward pass
    labels = labels.to(outputs.device)
    loss_fn(outputs, labels).backward()
    optimizer.step()

In [7]:
torch.cuda.synchronize()

import time
st=time.time()
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n")
    train(model)
    print('\n',time.time()-st)

Epoch 1

 1562
 34.15656566619873
Epoch 2

 1562
 68.22418117523193
Epoch 3

 1562
 101.9520435333252
